In [1]:
from dataset import CSQADataset

D:\!Cloud\Mega\Git\CARTON\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load data
dataset = CSQADataset()
vocabs = dataset.get_vocabs()
# train_data, val_data, _ = dataset.get_data()
# train_helper, val_helper, _ = dataset.get_data_helper()

In [3]:
print(len(vocabs['logical_form'].freqs))
print(vocabs['logical_form'].freqs)

25
Counter({'type': 38020, 'relation': 35313, 'entity': 29931, 'filter_type': 22179, 'find': 20257, 'find_reverse': 8268, 'count': 6556, 'union': 4857, 'find_reverse_tuple_counts': 3881, 'find_tuple_counts': 2907, 'is_in': 2440, 'value': 1357, 'intersection': 1343, 'approx': 1169, 'filter_multi_types': 1132, 'less': 1054, 'prev_answer': 1034, 'greater': 931, 'argmax': 348, 'argmin': 316, 'atmost': 303, 'atleast': 299, 'equal': 241, 'difference': 57, 'insert': 1})


In [4]:
vocabs['ner'].freqs

Counter({'O': 522594,
         'B-Q838948': 2707,
         'I-Q838948': 8644,
         'B-Q15617994': 8427,
         'I-Q15617994': 10103,
         'B-Q2221906': 336,
         'I-Q2221906': 307,
         'B-Q18608583': 98,
         'I-Q18608583': 334,
         'B-Q502895': 22447,
         'I-Q502895': 73452,
         'B-Q6619679': 205,
         'I-Q6619679': 479,
         'B-Q290': 1160,
         'B-Q56061': 4319,
         'B-Q62447': 109,
         'I-Q62447': 417,
         'B-Q18142': 56,
         'I-Q18142': 165,
         'B-Q43229': 794,
         'I-Q43229': 2517,
         'I-Q56061': 5270,
         'B-Q349': 844,
         'I-Q349': 777,
         'B-Q1076486': 140,
         'I-Q1076486': 460,
         'B-Q12737077': 3141,
         'I-Q12737077': 5797,
         'B-Q2342494': 457,
         'I-Q2342494': 2683,
         'B-Q34770': 1020,
         'B-Q34379': 945,
         'B-Q201658': 97,
         'I-Q201658': 325,
         'B-Q12973014': 424,
         'I-Q12973014': 1406,
         'B-Q

In [ ]:
vocabs['coref'].freqs

In [ ]:
vocabs['input'].freqs

In [ ]:
len(vocabs['predicate_pointer'].freqs)

In [ ]:
print([qt.value for qt in QuestionTypes])